In [1]:
import os
import glob
import time

import pandas as pd
import numpy as np
import xarray as xr
import datetime
from netCDF4 import Dataset


import matplotlib
from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter
matplotlib.rcParams.update({'font.size': 16})
%matplotlib inline
import cartopy.crs as ccrs
import cartopy

import sys
p = os.path.abspath('../..')
if p not in sys.path:
    sys.path.append(p)

from orographicPrecipitation.precip_model_functions import qsat,upslope_omega_sin
from orographicPrecipitation.precip_extremes_scaling import *
from wrfProcessing import *

import warnings
from tabulate import tabulate

In [15]:
def process(times):
    rep=[]
    i=0
    ti=time.time()
    for t in times:
        i+=1
        if i%1000==0:
            print(i)
        k=str(np.array(t.str.decode("utf-8")))
        rep.append(pd.to_datetime(k[:10]+" "+k[-8:]))
    print(time.time()-ti)
    return rep

def open_wrfds2(path):
    t=time.time()
    ds = xr.open_mfdataset(sorted(glob.glob(os.path.join(path,'wrfpout_d01_197*'))),combine="nested",concat_dim="Time")
    ds2 = xr.open_dataset(sorted(glob.glob(os.path.join(path,'wrfout_d01_197*')))[-1])
    ds['PSFC']=ds2['PSFC'].isel(Time=0).broadcast_like(ds['RAINC'])
    print(time.time()-t)
    ds=ds.rename({'num_press_levels_stag':'level'})
    newtime=process(ds["Times"].load())
    t=time.time()
    ds["time"]=(['Time'],  newtime)
    ds=ds.swap_dims({"Time":"time"})
    print(time.time()-t)
    return ds

class WrfDataset100km2 :
    def __init__(self, variables , lbl, sel_mountain):
        self.lbl = lbl
        self.sel_mountain=sel_mountain
        
        t=time.time()
        ds = variables
        self.vars = ds.chunk({'south_north': 35,'west_east': 40,'time':800})
        #number of outputs per day
        self.nhours = len(self.vars.time.sel(time = pd.to_datetime(np.array(self.vars.time.isel(time=0))).strftime("%Y-%m-%d")))
        print("loading time : %.1f s"%(time.time()-t))
        
        self.precip_g_daily = self.nhours*self.vars.RAINNC.isel(time=range(50*self.nhours,len(self.vars.time))).diff('time').assign_coords(time = self.vars.RAINNC.time.isel(time=range(50*self.nhours,len(self.vars.time)-1))).resample(time='1D').mean().compute()
        self.precip_c_daily = self.nhours*self.vars.RAINC.isel(time=range(50*self.nhours,len(self.vars.time))).diff('time').assign_coords(time = self.vars.RAINNC.time.isel(time=range(50*self.nhours,len(self.vars.time)-1))).resample(time='1D').mean().compute()
        self.precip_daily = self.precip_g_daily + self.precip_c_daily
        print("precips time : %.1f s"%(time.time()-t))
        
    def set_extreme_precip(self,quantile):
        self.ex_pr = self.precip_daily.chunk({'time': -1}).quantile(quantile,"time")
        self.ex_pr_zonmean = self.ex_pr.mean("west_east")
    def set_extreme_vars(self,quantile):
        self.ex_w,self.ex_t,self.ex_ps = extreme_vars(self.precip_daily,
                                                      self.vars.W_PL,
                                                      self.vars.T_PL,
                                                      self.vars.PSFC,
                                                      quantile
                                                     )
    def compute_ogscaling(self):
        self.ex_pr_og=ogscaling(self.ex_w,self.ex_t,self.ex_ps)
            
    def set_extreme_precip_mountain(self,quantile):
        mountain_precip = self.sel_mountain(self.precip_daily).mean(["south_north","west_east"])
        self.ex_pr_mountain = mountain_precip.chunk({'time': -1}).quantile(quantile,"time")
        self.ex_pr_mountain_all = mountain_precip.where(mountain_precip>self.ex_pr_mountain,drop=True)
    
    def set_extreme_vars_mountain(self,quantile):
        self.ex_w_mountain,self.ex_t_mountain,self.ex_ps_mountain = extreme_vars(self.sel_mountain(self.precip_daily).mean(["south_north","west_east"]),
                                                                                 self.sel_mountain(self.vars.W_PL  ),
                                                                                 self.sel_mountain(self.vars.T_PL  ),
                                                                                 self.sel_mountain(self.vars.PSFC),
                                                                                 quantile
                                                                                )   
    def compute_ogscaling_mountain(self):
        self.ex_pr_mountain_og=ogscaling(self.ex_w_mountain,self.ex_t_mountain,self.ex_ps_mountain).mean(["west_east","south_north"])
    
    def compute_ogscaling_mountain_allevents(self):
        #compute 6-hourly times corresponding to extreme events
        ex_times = [pd.to_datetime(d+' {:0>2}'.format(h)) for d in list(pd.to_datetime(np.array(self.ex_pr_mountain_all.time)).strftime("%Y-%m-%d")) for h in range(0,24,24//self.nhours)]
        ex_w_mountain_all  = self.sel_mountain(self.vars.W_PL.sel(time=ex_times)  )
        ex_t_mountain_all  = self.sel_mountain(self.vars.T_PL.sel(time=ex_times)  )
        ex_ps_mountain_all = self.sel_mountain(self.vars.PSFC.sel(time=ex_times))        
        self.ex_pr_mountain_og_all = ogscaling(ex_w_mountain_all,ex_t_mountain_all,ex_ps_mountain_all)
        self.ex_pr_mountain_og_all = self.ex_pr_mountain_og_all.resample(time='1D').mean(["time","west_east","south_north"] ).sel(time=self.ex_pr_mountain_all.time)

In [28]:
mcont

,Array,Chunk
Bytes,521.47 MB,5.38 MB
Shape,"(23280, 70, 80)","(240, 70, 80)"
Count,291 Tasks,97 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,521.47 MB,5.38 MB
Shape,"(23280, 70, 80)","(240, 70, 80)"
Count,291 Tasks,97 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [27]:
mcont.level=np.array([1000.,975.,950.,925.,900.,875.,850.,825.,800.,775.,750.,700.,650.,600.,550.,500.,450.,400.,350.,300.,250.,225.,200.,175.,150.,125.,100.,70.,50.,10.])

AttributeError: cannot set attribute 'level' on a 'Dataset' object. Use __setitem__ styleassignment (e.g., `ds['name'] = ...`) instead of assigning variables.

In [16]:
mcont=open_wrfds2('/global/cscratch1/sd/qnicolas/WRF/WRFV4control100km_half/test/em_beta_plane/')
mwarm=open_wrfds2('/global/cscratch1/sd/qnicolas/WRF/WRFV4control100km_half_warm/test/em_beta_plane/')

107.47054195404053
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
18.293818473815918
0.14207720756530762
3.81052565574646
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
15.37376356124878
0.13826608657836914


In [18]:
Mcont = WrfDataset100km2(mcont,"Mountain, control"       ,sel_tropicmountain)
Mwarm = WrfDataset100km2(mwarm,"Mountain, SST +2K, 2*CO2",sel_tropicmountain)

loading time : 6.4 s


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/resample.py:176: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, the dimension order of these coordinates will be restored as well unless you specify restore_coord_dims=False.
  super().__init__(*args, **kwargs)
/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/resample.py:176: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, the dimension order of these coordinates will be restored as well unless you specify restore_coord_dims=False.
  super().__init__(*args, **kwargs)


precips time : 671.9 s
loading time : 1.9 s


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/resample.py:176: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, the dimension order of these coordinates will be restored as well unless you specify restore_coord_dims=False.
  super().__init__(*args, **kwargs)
/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/xarray/core/resample.py:176: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, the dimension order of these coordinates will be restored as well unless you specify restore_coord_dims=False.
  super().__init__(*args, **kwargs)


precips time : 742.6 s


In [19]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore",FutureWarning)
    Acont = WrfDataset100km("/global/cscratch1/sd/qnicolas/wrfdata/processed","wrf.aquaplanet.100km.1000d"            ,"Aquaplanet, control"       ,sel_tropicmountain,open_ds=0)
    Awarm = WrfDataset100km("/global/cscratch1/sd/qnicolas/wrfdata/processed","wrf.aquaplanet.100km.SST2K_2XCO2.1000d","Aquaplanet, SST +2K, 2*CO2",sel_tropicmountain,open_ds=0)

loading time : 0.0 s
Daily precips already computed
precips time : 0.3 s
loading time : 0.0 s
Daily precips already computed
precips time : 0.4 s


In [20]:
def set_pctile(pctile):
    for ds in [Mcont,Mwarm,Acont,Awarm]:
        ds.set_extreme_precip(pctile/100)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore",RuntimeWarning)
            warnings.simplefilter("ignore",FutureWarning)
            ds.set_extreme_vars(pctile/100)
            ds.compute_ogscaling()

In [25]:
Mcont.path="$HOME/"
Mwarm.path="$HOME/"
Mcont.path="test"
Mwarm.path="test"

In [26]:
pctile=99
set_pctile(pctile)

KeyboardInterrupt: 

# Zonal-mean precip

In [ ]:
print("Mean precip increase, Mountain %.2f%%/K"%  (100*(spatial_mean(Mwarm.precip_daily.mean("time"))/spatial_mean(Mcont.precip_daily.mean("time")) - 1)))
print("Mean precip increase, Aquaplanet %.2f%%/K"%(100*(spatial_mean(Awarm.precip_daily.mean("time"))/spatial_mean(Acont.precip_daily.mean("time")) - 1)))


In [ ]:
fig, ax = plt.subplots(3,1,figsize=(15,3*6))
props = plt.rcParams['axes.prop_cycle'].by_key()['color']
i=0
for ds in [Mcont,Mwarm,Acont,Awarm]:
    ax[0].plot(ds.vars.south_north,ds.precip_daily.mean(["time","west_east"]),  label="{:>30}".format(ds.lbl)  ,color=props[i])
    ax[1].plot(ds.vars.south_north,ds.precip_g_daily.mean(["time","west_east"]),label="{:>30}".format(ds.lbl),color=props[i])
    ax[2].plot(ds.vars.south_north,ds.precip_c_daily.mean(["time","west_east"]),label="{:>30}".format(ds.lbl),color=props[i])
    i+=1

ax[0].set_title("Mean precip",fontsize=20)
ax[1].set_title("Mean Large-scale precip",fontsize=20)
ax[2].set_title("Mean Convective precip",fontsize=20)
ax[0].set_ylim(0,16)

for a in ax[:-1] :a.legend()

# Precipitation extremes

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(12,10))

for ds in [Mcont,Mwarm,Acont,Awarm]:
    ax[0].plot(ds.ex_pr.south_north,ds.ex_pr_zonmean,label=ds.lbl)
    ax[0].set_title("%i th percentile of precipitation"%pctile)
    ax[1].plot(ds.ex_pr.south_north,ds.ex_pr.sel(west_east=slice(38,40)).mean("west_east"),label=ds.lbl)
    ax[1].set_title("%i th percentile of precipitation, over mountain longitudes"%pctile)
plt.legend()

## Vertical motion profiles over tropics/mountains on precip extremes days

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(12,6))


with warnings.catch_warnings():
    warnings.simplefilter("ignore",category=RuntimeWarning)
    ax=ax1
    ax.plot(Mcont.ex_w.isel(south_north=range(5),west_east=range(38,40)).mean(["south_north","west_east"]),Mcont.ex_w.level,label="mountain, per grid cell",color="orange")
    ax.plot(Mwarm.ex_w.isel(south_north=range(5),west_east=range(38,40)).mean(["south_north","west_east"]),Mwarm.ex_w.level,label="mountain, per grid cell",color="red")

    ax.plot(Acont.ex_w.isel(south_north=range(5)).mean(["south_north","west_east"]),Acont.ex_w.level,label="rest of tropics, A",color="cyan")
    ax.plot(Awarm.ex_w.isel(south_north=range(5)).mean(["south_north","west_east"]),Awarm.ex_w.level,label="rest of tropics, A",color="blue")

    ax=ax2
    ax.plot(Mcont.ex_w.isel(south_north=range(5)).sel(west_east=((Mcont.ex_w.west_east<20)|(Mcont.ex_w.west_east>60))).mean(["south_north","west_east"]),Mcont.ex_w.level,label="rest of tropics, M",color="lawngreen")
    ax.plot(Mwarm.ex_w.isel(south_north=range(5)).sel(west_east=((Mwarm.ex_w.west_east<20)|(Mwarm.ex_w.west_east>60))).mean(["south_north","west_east"]),Mwarm.ex_w.level,label="rest of tropics, M",color="green")
    
    ax.plot(Acont.ex_w.isel(south_north=range(5)).mean(["south_north","west_east"]),Acont.ex_w.level,label="rest of tropics, A",color="cyan")
    ax.plot(Awarm.ex_w.isel(south_north=range(5)).mean(["south_north","west_east"]),Awarm.ex_w.level,label="rest of tropics, A",color="blue")

    
for ax in (ax1,ax2):
    ax.axvline(0,linestyle=':',color='k')
    ax.set_title("Vertical motion, %i th percentile precip"%pctile,fontsize=20)
    ax.legend(loc='lower right')
    ax.set_ylim(1010,0)
    



## Extreme precipitation sensitivity

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,5))
ax.plot(Acont.ex_pr.south_north,100*(Awarm.ex_pr_zonmean-Acont.ex_pr_zonmean)/Acont.ex_pr_zonmean/2, label = "Aquaplanet, %i th pctile"%pctile)
ax.plot(Mcont.ex_pr.south_north,100*(Mwarm.ex_pr_zonmean-Mcont.ex_pr_zonmean)/Mcont.ex_pr_zonmean/2, label = "Mountain, %i th pctile"%pctile)


with warnings.catch_warnings():
    warnings.simplefilter("ignore",category=FutureWarning)       
    resample_factor=10
    resampled_pr95_Mwarm = Mwarm.ex_pr.groupby(((Mwarm.ex_pr.south_north//resample_factor)+0.5) * resample_factor).mean('south_north').mean("west_east")
    resampled_pr95_Mcont = Mcont.ex_pr.groupby(((Mcont.ex_pr.south_north//resample_factor)+0.5) * resample_factor).mean('south_north').mean("west_east")
    resampled_pr95_Awarm = Awarm.ex_pr.groupby(((Awarm.ex_pr.south_north//resample_factor)+0.5) * resample_factor).mean('south_north').mean("west_east")
    resampled_pr95_Acont = Acont.ex_pr.groupby(((Acont.ex_pr.south_north//resample_factor)+0.5) * resample_factor).mean('south_north').mean("west_east")

ax.plot(resampled_pr95_Awarm.south_north,100*(resampled_pr95_Awarm-resampled_pr95_Acont)/resampled_pr95_Acont/2.,'-x',color='dodgerblue',label="bigger lat bins")
ax.plot(resampled_pr95_Mwarm.south_north,100*(resampled_pr95_Mwarm-resampled_pr95_Mcont)/resampled_pr95_Mcont/2.,'-x',color='orange',label="bigger lat bins")

ax.axhline(6.5,linestyle=":",label="CC scaling")
ax.axhline(0.,linestyle=":",color="k")

ax.set_ylim(-15,20)

plt.legend()
fig.suptitle("Sensitivity of the %i th pctile of precip, %%/(K of global mean temperature)"%pctile)

## Sensitivity breakdown (Fildier et al 2017)

In [ ]:
tropics_limit=5

def mean_mountain(x):
    return x.isel(south_north=range(5),west_east=range(38,40)).mean(['west_east','south_north'])
def mean_lee(x):
    return x.isel(south_north=range(5),west_east=range(40,42)).mean(['west_east','south_north'])
def mean_rest_tropics(x):
    return x.isel(south_north=range(tropics_limit)).sel(west_east=((x.west_east<20)|(x.west_east>60))).mean(['west_east','south_north'])

def mean_tropics(x):
    return x.isel(south_north=range(tropics_limit)).mean(['west_east','south_north'])
def mean_midlatitudes(x):
    return x.isel(south_north=range(40,60)).mean(['west_east','south_north'])

In [ ]:
def bsensitivity_breakdown(cont,warm,pctile,mean_tropics,mean_midlatitudes,mean_mountain,mean_lee,mountain=1):
    meanTinc=2
    with warnings.catch_warnings():
        warnings.simplefilter("ignore",category=RuntimeWarning)
                      
        prsensm  = bPR(cont,warm,meanTinc,mean_mountain    )
        prsensl  = bPR(cont,warm,meanTinc,mean_lee         )
        prsenst  = bPR(cont,warm,meanTinc,mean_tropics     )
        prsenss  = bPR(cont,warm,meanTinc,mean_subtropics  )
        prsense  = bPR(cont,warm,meanTinc,mean_midlatitudes)
        
        em  = bE(cont,warm,meanTinc,mean_mountain    )
        el  = bE(cont,warm,meanTinc,mean_lee         )
        et  = bE(cont,warm,meanTinc,mean_tropics     )
        es  = bE(cont,warm,meanTinc,mean_subtropics  )
        ee  = bE(cont,warm,meanTinc,mean_midlatitudes)
        
        d1m  = bD1(cont,warm,meanTinc,mean_mountain    )
        d1l  = bD1(cont,warm,meanTinc,mean_lee         )
        d1t  = bD1(cont,warm,meanTinc,mean_tropics     )
        d1s  = bD1(cont,warm,meanTinc,mean_subtropics  )
        d1e  = bD1(cont,warm,meanTinc,mean_midlatitudes)
        
        d2m  = bD2(cont,warm,meanTinc,mean_mountain    )
        d2l  = bD2(cont,warm,meanTinc,mean_lee         )
        d2t  = bD2(cont,warm,meanTinc,mean_tropics     )
        d2s  = bD2(cont,warm,meanTinc,mean_subtropics  )
        d2e  = bD2(cont,warm,meanTinc,mean_midlatitudes)
        
        thm  = bT(cont,warm,meanTinc,mean_mountain    )
        thl  = bT(cont,warm,meanTinc,mean_lee         )
        tht  = bT(cont,warm,meanTinc,mean_tropics     )
        ths  = bT(cont,warm,meanTinc,mean_subtropics  )
        the  = bT(cont,warm,meanTinc,mean_midlatitudes)
        
    print(' '*30+'\033[1m'+ "Extreme precip sensitivity breakdown, %i th percentile"%pctile + '\033[0m')
    
    tab=[['Tropics - 0°-%i°'%tropics_limit ,"%.2f"%(100*prsenst),"%.2f"%(100*et),"%.2f"%(100*d1t),"%.2f"%(100*d2t),"%.2f"%(100*tht),"%.2f"%(100*(prsenst-et-d1t-d2t-tht))],
         ['Midlatitudes - 40°-60°',         "%.2f"%(100*prsense),"%.2f"%(100*ee),"%.2f"%(100*d1e),"%.2f"%(100*d2e),"%.2f"%(100*the),"%.2f"%(100*(prsense-ee-d1e-d2e-the))],
        ]
    if mountain :
        tab += [['Mountain - Windward slope',"%.2f"%(100*prsensm),"%.2f"%(100*em),"%.2f"%(100*d1m),"%.2f"%(100*d2m),"%.2f"%(100*thm),"%.2f"%(100*(prsensm-em-d1m-d2m-thm))],
                ['Mountain - Lee',           "%.2f"%(100*prsensl),"%.2f"%(100*el),"%.2f"%(100*d1l),"%.2f"%(100*d2l),"%.2f"%(100*thl),"%.2f"%(100*(prsensl-el-d1l-d2l-thl))],
               ]
    
    print(tabulate(tab, headers=['Region', 'Precip (%)', 'Efficiency (%)', 'D1 (%)', 'D2 (%)', 'T (%)', 'N (%)']))


In [ ]:
print("Tropics Mountain")
t=time.time()
bsensitivity_breakdown(Mcont,Mwarm,pctile,mean_rest_tropics,mean_midlatitudes,mean_mountain,mean_lee,mountain=1)
print(time.time()-t)
print("Aquaplanet")
t=time.time()
bsensitivity_breakdown(Acont,Awarm,pctile,mean_tropics,mean_midlatitudes,mean_mountain,mean_lee,mountain=0)
print(time.time()-t)

In [ ]:
pctile=95
set_pctile(pctile)
print("Tropics Mountain")
t=time.time()
bsensitivity_breakdown(Mcont,Mwarm,pctile,mean_rest_tropics,mean_midlatitudes,mean_mountain,mean_lee,mountain=1)
print(time.time()-t)
print("Aquaplanet")
t=time.time()
bsensitivity_breakdown(Acont,Awarm,pctile,mean_tropics,mean_midlatitudes,mean_mountain,mean_lee,mountain=0)
print(time.time()-t)